<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training4/blob/main/Words2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/wiki_zh_2019
#unzip the zipfile
!pip install opencc-python-reimplemente
!pip install jieba
!pip install word2vec

In [23]:
from os import listdir
from os.path import isfile, isdir, join
import json
from opencc import OpenCC
filesList = {
    0:"AA",
    1:"AB",
    2:"AC",
    4:"AD",
    5:"AE",
    6:"AF",
    7:"AG",
    8:"AH",    
    9:"AI",
    10:"AJ",
    11:"AK",
    12:"AL",
    13:"AM"
}

def loadData():
  ALL_DATAS = []
  for i in filesList:
    folderPath = "./wiki_zh/%s/" % filesList[i]
    files = listdir(folderPath)
    for i in files:
      curFilePath = folderPath + "%s"%i 
      print(curFilePath)
      with open(curFilePath) as f:
        for j in f.readlines():
          str2Json = json.loads(j)
          text = "".join(str2Json["text"].strip().split())
          ALL_DATAS.append(text)
        f.close()
  chineseSimpTotrad(ALL_DATAS)

def chineseSimpTotrad(data):
  ccConvertor = OpenCC("s2hk")
  transChineseList = []
  for i in data:
    transChineseList.append(ccConvertor.convert(i))
  return transChineseList


# cut words

In [32]:
#load stopwords list
import codecs
import jieba
import jieba.analyse

def wordCutting(wordList,stopWordlist):
  cutWordsListDatas = []
  for i in wordList:
      cutWordsListDatas.append(" ".join([word for word in jieba.cut(i,cut_all=False) if word not in stopWordlist]))
  return cutWordsListDatas


# Word2vec for our text

In [28]:
import word2vec

def modelGenerate(cutWordTxt):
  word2vec.word2vec(cutWordTxt,"demo.bin",size=300,verbose=True,binary=True)
  print("Done....")

def loadModel(binFielPath):
  return word2vec.load(binFielPath)

In [38]:
if __name__ == '__main__':

  #load All data and convert to traditional chinese
  chineseWordList = loadData()

  #load stop word list
  fp = codecs.open("./stopwords.dat","r",encoding="utf-8")
  contents = fp.read()
  fp.close()

  # word cutting and stoplist
  cutWordsListDatas = wordCutting(chineseWordList,contents)

  # #store to a txt
  cutWordsTxt = "cutWordsList2.txt"
  with open(cutWordsTxt,"w",encoding = "utf-8") as fw:
    for data in cutWordsListDatas:
      fw.write(data)
      fw.write("\n")

  modelGenerate(cutWordsTxt)
  model = loadModel("/content/drive/MyDrive/demo.bin")
  predictWord = "鋼鐵人"
  datas = model.similar(predictWord,n=20)
  word = list(datas[0])
  score = list(datas[1])
  for i in range(len(word)):
      print("%s : %s" % (model.vocab[word[i]],score[i]))


Running command: word2vec -train cutWordsList2.txt -output demo.bin -size 300 -window 5 -sample 1e-3 -hs 0 -negative 5 -threads 12 -iter 5 -min-count 5 -alpha 0.025 -debug 2 -binary 1 -cbow 1
Starting training using file cutWordsList2.txt
Vocab size: 1
Words in train file: 1
Done....
